In [72]:
import mne
import os

#Find all the files in the data/Alken-29March/ folder with .dat extension  
files = [f for f in os.listdir('data/Alken-29March/') if f.endswith('.dat')]
#Sort the files in ascending order
files.sort()
#Removes the test file
files.pop()
#Read the files into a list of raw objects
raws = [mne.io.read_raw_curry('data/Alken-29March/' + f, preload=True) for f in files]



Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 139799  =      0.000 ...    69.900 secs...
Event file not found. No Annotations set.
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 193399  =      0.000 ...    96.700 secs...
Event file not found. No Annotations set.
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 128799  =      0.000 ...    64.400 secs...
Event file not found. No Annotations set.
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 125199  =      0.000 ...    62.599 secs...
Event file not found. No Annotations set.
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 133799  =      0.000 ...    66.900 secs...
Event file not found. No Annotations set.
Leaving device<->head transform as None (no landmarks found)
Reading 0 ... 166999  =      0.000 ...    83.499 secs...
Event file not found. No Annotations set.
Leaving device<->head transform as None 